In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'tube_size_metric'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
field='TubeSize'
fields = [field]
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

2274


In [5]:
# df[field].explode().value_counts()
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, TubeSize]
Index: []

In [6]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [7]:
df[field].explode().value_counts()

3/8"           350
1/2"           330
5/8"           284
3/4"           278
1/4"           219
1"             178
5/16"          114
1-1/4"          94
7/8"            23
1-1/2"          15
7/16"           13
3/16"           13
2"               9
1/8"             4
1/4" - 5/8"      4
1/4" - 1/2"      1
Name: TubeSize, dtype: int64

In [17]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)(tube)|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(?i)(\d+.{0,2}mm)|()'
df['matches']=df[field].apply(lambda x: re_extract(pat, str(x)))

pat=r'(?i)(m)|()'
df['mm']=df[field].apply(lambda x: re_extract(pat, str(x)))


custom=df[(df[field].astype(str)!='nan')]
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x))).apply(lambda x: re.sub(r'(?i)\s?mm',' mm',str(x))).apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r'\.0','',str(x))).apply(lambda x: re.sub(r'(?i)\s?Am?p?s?','A',str(x)))#.apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
# custom[curation_col]=custom[field].apply(lambda x: f'{x}"').apply(lambda x: re.sub(r'""','"',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
matchcustom[curation_col]=matchcustom[curation_col].apply(lambda x: re.sub(r'"?\s?x\s?','" - ',str(x)))
matchcustom=matchcustom[matchcustom[curation_col].astype(str)!='']
matchcustom[curation_col].explode().value_counts()
custom

1929


buckets external_id  \
0     Compressors Accessories & Replacement Parts      405783   
1     Compressors Accessories & Replacement Parts      405784   
2     Compressors Accessories & Replacement Parts      405785   
3     Compressors Accessories & Replacement Parts      405786   
4     Compressors Accessories & Replacement Parts      405787   
...                                           ...         ...   
2088  Compressors Accessories & Replacement Parts     50008WH   
2089  Compressors Accessories & Replacement Parts  50020X16WH   
2090  Compressors Accessories & Replacement Parts     50032WH   
2091  Compressors Accessories & Replacement Parts  50008X12WH   
2092  Compressors Accessories & Replacement Parts     60024WH   

                                                     product_name  \
0     1/4 - 1/2 Kim-Krimp "Z" Series Face-Seal Fitting Assortment   
1     1/4 - 5/8 Kim-Krimp "Z" Series Face-Seal Fitting Assortment   
2               1/4 - 1/2 Kim-Krimp "Z" Series Fitting Assortment   
3               1/4 - 1/2 Kim-Krimp "Z" Series Fitting Assortment   
4        1/4 - 1/2 Kim-Krimp "Z" Series O-Ring Fitting Assortment   
...                                                           ...   
2088                                   1/2 x 1/2 Straight Adapter   
2089                                  1-1/4 x 1" Straight Adapter   
2090                                     2" x 2" Straight Adapter   
2091                                   1/2 x 3/4 Straight Adapter   
2092                               1-1/2 x 1-1/2 Straight Adapter   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        long_desc  \
0     TYPICAL APPLICATION: General Purpose high pressure hydraulics. COMPATIBLE HOSES: DK145, K245, DK245, K425, DK425, K280. PRESSURE: Determined by maximum working pressure for hose size. MATERIAL: Low carbon steel. PLATING: Trivalent zinc. ADVANTAGES: Wide selection of hose and end configurations allowing for a diverse number of applications. ASSEMBLE WITH: Kim-Krimp K-Series and Weatherhead Coll-O-Crimp® Equipment. Weatherhead® and Coll-O-Crimp® are registered trademarks of Eaton Corporation. Eaton's warranty covers only hose assemblies made using Eaton Weatherhead® Hose Assembly Crimp Machines, crimp dies, hose products and hose fittings. Kimball Midwest's warranty covers our Kim-Krimp line, including its compatibility with with various Weatherhead® products, when used as specified by Kimball Midwest.   
1     TYPICAL APPLICATION: General Purpose high pressure hydraulics. COMPATIBLE HOSES: DK145, K245, DK245, K425, DK425, K280. PRESSURE: Determined by maximum working pressure for hose size. MATERIAL: Low carbon steel. PLATING: Trivalent zinc. ADVANTAGES: Wide selection of hose and end configurations allowing for a diverse number of applications. ASSEMBLE WITH: Kim-Krimp K-Series and Weatherhead Coll-O-Crimp® Equipment. Weatherhead® and Coll-O-Crimp® are registered trademarks of Eaton Corporation. Eaton's warranty covers only hose assemblies made using Eaton Weatherhead® Hose Assembly Crimp Machines, crimp dies, hose products and hose fittings. Kimball Midwest's warranty covers our Kim-Krimp line, including its compatibility with with various Weatherhead® products, when used as sp

In [9]:
# not_match=df[(df[field].astype(str)=='nan')]
# # print(len(not_match[(not_match['ld'].astype(str)=='[]')|(not_match['name'].astype(str)!='[]')]))
# two=not_match[(not_match['ld'].astype(str)!='[]')|(not_match['name'].astype(str)!='[]')]
# # print(len(two))
# # two[curation_col]=two['name'].apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))


# two[curation_col]=two['ld'].apply(lambda x: re.sub(r'(?i)\s?amp','A',str(x)))
# matchtwo=two[['external_id',curation_col]]
# print(len(matchtwo))
# two

In [10]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [11]:
# two=df[(df['matches'].astype(str)=='[]')]
# print(len(two[two['name'].astype(str)!='[]']))
# two=two[two['name'].astype(str)!='[]']
# two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
# two[curation_col].explode().value_counts()
# matchtwo=two[['external_id',curation_col]]

In [12]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [18]:
na=df[(df['name'].astype(str)=='[]')&(df['ld'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))
na

2031


buckets external_id  \
0     Compressors Accessories & Replacement Parts      405783   
1     Compressors Accessories & Replacement Parts      405784   
2     Compressors Accessories & Replacement Parts      405785   
3     Compressors Accessories & Replacement Parts      405786   
4     Compressors Accessories & Replacement Parts      405787   
...                                           ...         ...   
2269  Compressors Accessories & Replacement Parts       51349   
2270  Compressors Accessories & Replacement Parts       51394   
2271  Compressors Accessories & Replacement Parts       51391   
2272  Compressors Accessories & Replacement Parts       51397   
2273  Compressors Accessories & Replacement Parts       51504   

                                                     product_name  \
0     1/4 - 1/2 Kim-Krimp "Z" Series Face-Seal Fitting Assortment   
1     1/4 - 5/8 Kim-Krimp "Z" Series Face-Seal Fitting Assortment   
2               1/4 - 1/2 Kim-Krimp "Z" Series Fitting Assortment   
3               1/4 - 1/2 Kim-Krimp "Z" Series Fitting Assortment   
4        1/4 - 1/2 Kim-Krimp "Z" Series O-Ring Fitting Assortment   
...                                                           ...   
2269                                     5mm x 1.0mm Viton O-Ring   
2270                                    25mm x 2.5mm Viton O-Ring   
2271                                    22mm x 2.5mm Viton O-Ring   
2272                                    11mm x 3.0mm Viton O-Ring   
2273                                    37mm x 4.0mm Viton O-Ring   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        long_desc  \
0     TYPICAL APPLICATION: General Purpose high pressure hydraulics. COMPATIBLE HOSES: DK145, K245, DK245, K425, DK425, K280. PRESSURE: Determined by maximum working pressure for hose size. MATERIAL: Low carbon steel. PLATING: Trivalent zinc. ADVANTAGES: Wide selection of hose and end configurations allowing for a diverse number of applications. ASSEMBLE WITH: Kim-Krimp K-Series and Weatherhead Coll-O-Crimp® Equipment. Weatherhead® and Coll-O-Crimp® are registered trademarks of Eaton Corporation. Eaton's warranty covers only hose assemblies made using Eaton Weatherhead® Hose Assembly Crimp Machines, crimp dies, hose products and hose fittings. Kimball Midwest's warranty covers our Kim-Krimp line, including its compatibility with with various Weatherhead® products, when used as specified by Kimball Midwest.   
1     TYPICAL APPLICATION: General Purpose high pressure hydraulics. COMPATIBLE HOSES: DK145, K245, DK245, K425, DK425, K280. PRESSURE: Determined by maximum working pressure for hose size. MATERIAL: Low carbon steel. PLATING: Trivalent zinc. ADVANTAGES: Wide selection of hose and end configurations allowing for a diverse number of applications. ASSEMBLE WITH: Kim-Krimp K-Series and Weatherhead Coll-O-Crimp® Equipment. Weatherhead® and Coll-O-Crimp® are registered trademarks of Eaton Corporation. Eaton's warranty covers only hose assemblies made using Eaton Weatherhead® Hose Assembly Crimp Machines, crimp dies, hose products and hose fittings. Kimball Midwest's warranty covers our Kim-Krimp line, including its compatibility with with various Weatherhead® products, when used as sp

In [14]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [15]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [19]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 